In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import numpy as np
from model import ResNet

2025-03-09 08:39:00.014931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
log_dir = "logs/fit/plain56"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
pixel_mean = np.mean(x_train, axis=0)

x_train = x_train - pixel_mean
x_test = x_test - pixel_mean

In [ ]:
x_train = tf.keras.layers.ZeroPadding2D(padding=(4, 4))(x_train)

data_argumentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode='horizontal', seed=42),
    tf.keras.layers.RandomCrop(32, 32, seed=42, name='random_crop')
])

In [ ]:
resnet = ResNet(n=9, is_plain=True)

Successfully created 56-layer PlainNet


In [10]:
cifar_resnet = tf.keras.Sequential([
    data_argumentation,
    resnet
])

In [ ]:
class AccuracyError(tf.keras.metrics.Metric):
    def __init__(self, name="accuracy_error", **kwargs):
        super(AccuracyError, self).__init__(name=name, **kwargs)
        self.accuracy = tf.keras.metrics.Accuracy()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1) 
        self.accuracy.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return 1.0 - self.accuracy.result()

    def reset_state(self):
        self.accuracy.reset_state()

In [ ]:
sgd_optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.1, momentum=0.9, weight_decay=0.0001
)

cifar_resnet.compile(
               optimizer=sgd_optimizer,
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy', AccuracyError()]
)

In [13]:
def lr_scheduler(epoch, lr):
    iter_to_epoch = {'32k': 80, '48k': 122}
    if epoch in [iter_to_epoch['32k'], iter_to_epoch['48k']]:
        lr /= 10
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [14]:
cifar_resnet.fit(
    x_train, y_train,
    batch_size=128,
    epochs=160,
    callbacks=[lr_callback, tensorboard_callback]
)

Epoch 1/160


2025-03-09 08:40:26.738084: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


391/391 ━━━━━━━━━━━━━━━━━━━━ 64s 107ms/step - accuracy: 0.1293 - accuracy_error: 0.8707 - loss: 2.2671 - learning_rate: 0.1000
Epoch 2/160
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 106ms/step - accuracy: 0.2229 - accuracy_error: 0.7771 - loss: 2.0428 - learning_rate: 0.1000
Epoch 3/160
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 107ms/step - accuracy: 0.2625 - accuracy_error: 0.7375 - loss: 1.9470 - learning_rate: 0.1000
Epoch 4/160
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.3088 - accuracy_error: 0.6912 - loss: 1.8403 - learning_rate: 0.1000
Epoch 5/160
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.3423 - accuracy_error: 0.6577 - loss: 1.7503 - learning_rate: 0.1000
Epoch 6/160
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.3625 - accuracy_error: 0.6375 - loss: 1.7007 - learning_rate: 0.1000
Epoch 7/160
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 106ms/step - accuracy: 0.3803 - accuracy_error: 0.6197 - loss: 1.6551 - learning_rate: 0.1000
Epoch 8/160
391/391 ━━━━━━━━━━━━━━━━━━━

In [16]:
cifar_resnet.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8517 - accuracy_error: 0.1483 - loss: 0.4568


[0.46245792508125305, 0.8511999845504761, 0.14880001544952393]

In [ ]:
%tensorboard --logdir logs/fit/